# Analyse des clients du site d'e-commerce Olist

In [1]:
"""%load_ext pycodestyle_magic
%pycodestyle_on"""

In [2]:
# Importation des librairies nécessaires
import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import gc
import datetime
from function_utils import haversine_vectorize

9:1: E303 too many blank lines (3)
9:1: E402 module level import not at top of file
9:20: W291 trailing whitespace
11:1: E402 module level import not at top of file
11:19: W291 trailing whitespace
13:1: E402 module level import not at top of file
13:32: W291 trailing whitespace
15:1: E402 module level import not at top of file
17:1: E402 module level import not at top of file
19:1: E402 module level import not at top of file
21:1: E402 module level import not at top of file


In [3]:
sns.set_palette("Pastel2")

## 1) Création d'une segmentation RFM

### Information clients

In [4]:
# Importation des données clients
df_customers = pd.read_csv("datasets/olist_customers_dataset.csv")
df_geo = pd.read_csv("datasets/olist_geolocation_dataset.csv")

In [5]:
# Ajout des données de géolocalisation pour chaque client
df_customers = pd.merge(df_geo, df_customers,
                        right_on=["customer_zip_code_prefix", "customer_city", "customer_state"], 
                        left_on=["geolocation_zip_code_prefix", "geolocation_city", "geolocation_state"], how="right")
df_customers.drop(["geolocation_zip_code_prefix", "geolocation_city",
                   "geolocation_state"], axis=1, inplace=True)
df_customers = df_customers[['customer_id', "customer_unique_id", 
                             "customer_city", "geolocation_lat", "geolocation_lng"]]
df_customers.columns = ["customer_id", "customer_unique_id", 
                        "customer_city", "customer_latitude", "customer_longitude"]
df_customers.head()

,customer_id,customer_unique_id,customer_city,customer_latitude,customer_longitude
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,franca,-20.509897,-47.397866
1,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,franca,-20.497396,-47.399241
2,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,franca,-20.510459,-47.399553
3,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,franca,-20.480940,-47.394161
4,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,franca,-20.515413,-47.398194


5:80: E501 line too long (97 > 79 characters)
5:98: W291 trailing whitespace
7:80: E501 line too long (118 > 79 characters)
9:80: E501 line too long (113 > 79 characters)
11:80: E501 line too long (121 > 79 characters)
13:80: E501 line too long (120 > 79 characters)


In [6]:
print("Nombre d'id unique:", df_customers["customer_unique_id"].nunique())

Nombre d'id unique: 96096


In [7]:
print("Nombre d'id clients:", df_customers["customer_id"].nunique())

Nombre d'id clients: 99441


On remarque un écart important entre les identifiants clients et les identifiants uniques. On va donc supprimer les doublons sur ces deux colonnes.

In [8]:
# Suppression des doublons d'identifiant clients
df_customers.drop_duplicates(subset="customer_id", inplace=True)

In [9]:
df_customers.shape

(99441, 5)

### Information sur les commandes

In [10]:
# Importation des informations sur les commandes
df_orders = pd.read_csv("datasets/olist_orders_dataset.csv")
df_orders.shape

(99441, 8)

In [11]:
# Association entre commande et clients pour calculer la récence et la fréquence d'achat
df = pd.merge(df_customers, df_orders, 
              left_on="customer_id", right_on="customer_id", how="right")
df.drop_duplicates(subset="customer_id", 
                   inplace=True)
df.shape

(99441, 12)

1:80: E501 line too long (88 > 79 characters)
3:80: E501 line too long (98 > 79 characters)


In [12]:
# Importation des informations sur le paiement des commandes
df_orders_payments = pd.read_csv("datasets/olist_order_payments_dataset.csv")
df_orders_payments.shape

(103886, 5)

In [13]:
# Association entre commande et paiements 
# pour calculer la valeur monnaitaire d'un client
df = pd.merge(df, df_orders_payments, left_on="order_id", 
              right_on="order_id", how="right")
df.shape

(103886, 16)

1:80: E501 line too long (89 > 79 characters)
2:80: E501 line too long (91 > 79 characters)


In [14]:
df.head()

,customer_id,customer_unique_id,customer_city,customer_latitude,customer_longitude,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value
0,0a8556ac6be836b46b3e89920d59291c,708ab75d2a007f0564aedd11139c7708,teofilo otoni,-17.851690,-41.493610,b81ef226f3fe1789b1e8b2acac839d17,delivered,2018-04-25 22:01:49,2018-04-25 22:15:09,2018-05-02 15:20:00,2018-05-09 17:36:51,2018-05-22 00:00:00,1,credit_card,8,99.33
1,f2c7fc58a9de810828715166c672f10a,a8b9d3a27068454b1c98cc67d4e31e6f,sao paulo,-23.470015,-46.634846,a9810da82917af2d9aefd1278f1dcfa0,delivered,2018-06-26 11:01:38,2018-06-26 11:18:58,2018-06-28 14:18:00,2018-06-29 20:32:09,2018-07-16 00:00:00,1,credit_card,1,24.39
2,25b14b69de0b6e184ae6fe2755e478f9,6f70c0b2f7552832ba46eb57b1c5651e,sao paulo,-23.454897,-46.656832,25e8ea4e93396b6fa0d3dd708e76c1bd,delivered,2017-12-12 11:19:55,2017-12-14 09:52:34,2017-12-15 20:13:22,2017-12-18 17:24:41,2018-01-04 00:00:00,1,credit_card,1,65.71
3,7a5d8efaaa1081f800628c30d2b0728f,87695ed086ebd36f20404c82d20fca87,juiz de fora,-21.750376,-43.342449,ba78997921bbcdc1373bb41e913ab953,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,2017-12-21 01:35:51,2018-01-04 00:00:00,1,credit_card,8,107.78
4,15fd6fb8f8312dbb4674e4518d6fa3b3,4291db0da71914754618cd789aebcd56,conchas,-23.015379,-48.008990,42fdf880ba16b47b59251dd489d4441a,delivered,2018-05-21 13:59:17,2018-05-21 16:14:41,2018-05-22 11:46:00,2018-06-01 21:44:53,2018-06-13 00:00:00,1,credit_card,2,128.45


Pour calculer la récence, on va transformer les éléments en lien avec les dates en format datetime. Ensuite, on va calculer le nombre de jours entre le dernier client à avoir acheté +1 que l'on va soustraire à la dernière date d'achat de chaque client.

In [15]:
# Transformation des timestamps en format datetime
time = ["order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date", "order_delivered_customer_date", "order_estimated_delivery_date"]

for cols in time:
    df[cols] = pd.to_datetime(df[cols], format="%Y-%m-%d", errors="ignore")

2:80: E501 line too long (154 > 79 characters)
6:1: E303 too many blank lines (3)


In [16]:
snapshot_date = max(df.order_purchase_timestamp) + datetime.timedelta(days=1)

In [19]:
rfm_custom = df.groupby("customer_unique_id").agg({
    "order_purchase_timestamp" : lambda x: (snapshot_date - x.max()).days,
    "order_id" : 'count',
    "payment_value" : "mean"
})

SyntaxError: unexpected character after line continuation character (<ipython-input-19-55254936f6a0>, line 2)

2:5: E128 continuation line under-indented for visual indent


ValueError: too many values to unpack (expected 3)

In [ ]:
rfm_custom.columns = ["Recency", "Frequency", "MonetaryValue"]
data = rfm_custom.copy().reset_index()

In [ ]:
data.head()

In [ ]:
data['r_quartile'] = pd.qcut(data['Recency'], 4, ['1','2','3','4'])
data['f_bins'] = pd.cut(data['Frequency'], 2, labels=['2','1'])
data['m_quartile'] = pd.qcut(data['MonetaryValue'], 4, ['4','3','2','1'])

In [ ]:
data['RFM_Score'] = data["r_quartile"].astype(str)+ data["f_bins"].astype(str) + data["m_quartile"].astype(str)
data.head(3)

In [ ]:
data.sort_values("RFM_Score", ascending=True)

### Analyse de la segmentation RFM

In [ ]:
# Statistiques descriptives
data.describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharex=True)
fig.suptitle('Description des montants dépensés par les clients')

sns.histplot(ax=axes[0],x="MonetaryValue", data=data, kde=True)
axes[0].set_title("Distribution des montants dépensés")

sns.boxplot(ax=axes[1], x="MonetaryValue", data=data)
axes[1].set_title("Boxplot des montants dépensés")

plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharex=True)
fig.suptitle('Description des jours passés depuis la dernière commande')

sns.histplot(ax=axes[0],x="Recency", data=data, kde=True)
axes[0].set_title("Distribution des jours passés depuis la dernière commande")

sns.boxplot(ax=axes[1], x="Recency", data=data)
axes[1].set_title("Boxplot des jours passés depuis la dernière commande")

plt.show()

In [ ]:
sns.countplot(y="Frequency", data=data)
plt.ylim(23)
plt.title("Répartition de la fréquence d'achat")

plt.show()

La fréquence et la récence ne semblent pas être très discriminants. En effet, la plupart des clients réalise un seul achat. De même, il est difficile de savoir s'ils reviennent régulièrement acheter donc la récence est très variable. 

En inspectant le fonctionnement du site, il s'avère que l'on peut commander sans avoir besoin de créer un compte. On ne peut donc pas savoir si une personne commande plusieurs fois et obtenir les informations nécessaires. 

In [ ]:
del df_orders_payments
del rfm_custom

In [ ]:
gc.collect()

In [ ]:
data.set_index("customer_unique_id", inplace=True)

In [ ]:
# Enregistrement des données de la segmentation
data.to_csv("datasets/data_segmentation_rfm.csv")

## 2) Ajout de variables supplémentaires

On va créer de nouvelles variables que l'on va ajouter pour pouvoir obtenir une segmentation plus fine.

In [ ]:
data.reset_index(inplace=True)

### Calcul des distances entre vendeurs et acheteurs

In [ ]:
# Importation des informations vendeurs

df_sellers = pd.read_csv("datasets/olist_sellers_dataset.csv")
df_sellers.head(3)

In [ ]:
df_sellers = pd.merge(df_geo, df_sellers,
                        right_on=["seller_zip_code_prefix", "seller_city", "seller_state"], 
                        left_on=["geolocation_zip_code_prefix", "geolocation_city", "geolocation_state"], how="right")
df_sellers.drop(["geolocation_zip_code_prefix", "geolocation_city", "geolocation_state"], axis=1, inplace=True)
df_sellers = df_sellers[['seller_id', "geolocation_lat", "geolocation_lng"]]
df_sellers.columns = ["seller_id", "seller_latitude", "seller_longitude"]
df_sellers.head()

In [ ]:
df_sellers.drop_duplicates(subset="seller_id", inplace=True)

In [ ]:
# Importation des informations sur les commandes

df_items = pd.read_csv("datasets/olist_order_items_dataset.csv")
df_items.head(3)

In [ ]:
df_items = pd.merge(df_orders, df_items, on="order_id", how="left")

In [ ]:
df_items = pd.merge(df_items, df_customers, on="customer_id", how="left")

In [ ]:
df_items = pd.merge(df_sellers, df_items, on="seller_id", how="left")

In [ ]:
df_items.dropna(subset=["customer_city"], inplace=True)

In [ ]:
df_items["distance_customer_seller"] = haversine_vectorize(df_items["customer_longitude"], df_items["customer_latitude"], df_items["seller_longitude"], df_items["seller_latitude"])

In [ ]:
df_items = df_items[["customer_unique_id", "product_id", "distance_customer_seller"]]
data = pd.merge(data, df_items, on="customer_unique_id", how="left")

### Informations sur le type de produits commandés

In [ ]:
df_products = pd.read_csv("datasets/olist_products_dataset.csv")
df_translation = pd.read_csv("datasets/product_category_name_translation.csv")
df_products = pd.merge(df_products, df_translation, on="product_category_name")
df_products = df_products[["product_id", "product_category_name_english"]]
df_products.rename(columns={"product_category_name_english" : "product_category_name"}, inplace=True)
df_products.head()

In [ ]:
data = pd.merge(data, df_products, on="product_id", how="left")
data.drop("product_id", axis=1, inplace=True)

### Informations sur les avis clients

In [ ]:
df_reviews = pd.read_csv("datasets/olist_order_reviews_dataset.csv")
df_reviews.head(3)

In [ ]:
df_reviews.drop(["review_id", "review_creation_date", "review_answer_timestamp"], axis=1, inplace=True)

In [ ]:
df_reviews["review_score"] = df_reviews["review_score"].astype("int32")

In [ ]:
df_orders = pd.read_csv("datasets/olist_orders_dataset.csv")
df_orders.head(3)

In [ ]:
df_orders.drop(["order_purchase_timestamp", "order_delivered_carrier_date"], axis=1, inplace=True)
cols_time = ["order_approved_at", "order_delivered_customer_date", "order_estimated_delivery_date"]

for cols in cols_time:
    df_orders[cols] = pd.to_datetime(df_orders[cols], format="%Y-%m-%d", errors="ignore")

In [ ]:
df_orders.info()

### Calcul entre du nombre de jours entre la commande et la réception réel et estimé

In [ ]:
df_orders["nb_days_before_delivered"] = df_orders["order_delivered_customer_date"] - df_orders["order_approved_at"]
df_orders["nb_days_before_delivered_estimation"] = df_orders["order_estimated_delivery_date"] - df_orders["order_approved_at"]

cols_var = ["nb_days_before_delivered", "nb_days_before_delivered_estimation"]

for cols in cols_var:
    df_orders[cols] = df_orders[cols].dt.days

df_orders.drop(["order_approved_at", "order_delivered_customer_date", "order_estimated_delivery_date"], axis=1, inplace=True)

In [ ]:
df_orders.head()

### Assemblage des différentes variables pour obtenir un dataset unique

In [ ]:
df_customers_orders = pd.merge(df_customers, df_orders, on="customer_id", how="left")
df_customers_orders.shape

In [ ]:
df_customers_orders = pd.merge(df_customers_orders, df_reviews, on="order_id", how="left")
df_customers_orders.shape

In [ ]:
df_customers_orders.head(3)

In [ ]:
df_customers_orders.drop(["order_id"], axis=1, inplace=True)

In [ ]:
df_customers_orders.shape

In [ ]:
df_customers_orders = df_customers_orders.groupby("customer_unique_id").mean().reset_index()

In [ ]:
data = pd.merge(data, df_customers_orders, on="customer_unique_id", how="left")
data.shape

In [ ]:
data = pd.merge(df_customers, data, on="customer_unique_id", how="right")
data.shape

In [ ]:
df_customers.head()

In [ ]:
data.drop_duplicates(subset="customer_unique_id", inplace=True)

In [ ]:
data.drop(["customer_id","customer_longitude_x", "customer_latitude_x", "customer_longitude_y", "customer_latitude_y"], axis=1, inplace=True)
data.set_index("customer_unique_id", inplace=True)
data.head(3)

In [ ]:
data = data.loc[data["nb_days_before_delivered"] > 0]
data = data.loc[data["nb_days_before_delivered_estimation"] > 0]
data = data.loc[data["distance_customer_seller"] > 0]

In [ ]:
data["review_score"] = data["review_score"].astype("int32")

## 3) Analyse des données clients

In [ ]:
# statistiques descriptives
data.describe()

### Analyses univariées des variables utilisées pour la segmentation

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('Description du nombre de jours entre la commande et la réception (réalité)')

sns.histplot(ax=axes[0],x="nb_days_before_delivered", data=data, kde=True)
axes[0].set_title("Distribution du nombre de jours entre la commande et la réception (réalité)")

sns.boxplot(ax=axes[1], x="nb_days_before_delivered", data=data)
axes[1].set_title("Boxplot du nombre de jours entre la commande et la réception (réalité)")

plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharex=True)
fig.suptitle('Description du nombre de jours entre la commande et la réception (estimation)')

sns.histplot(ax=axes[0],x="nb_days_before_delivered_estimation", data=data, kde=True)
axes[0].set_title("Distribution du nombre de jours entre la commande et la réception (estimation)")

sns.boxplot(ax=axes[1], x="nb_days_before_delivered_estimation", data=data)
axes[1].set_title("Boxplot du nombre de jours entre la commande et la réception (estimation)")

plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharex=False)
fig.suptitle('Description des avis clients')

sns.countplot(ax=axes[0],y="review_score", data=data)
axes[0].set_title("Distribution des avis clients")

sns.boxplot(ax=axes[1], x="review_score", data=data)
axes[1].set_title("Boxplot des avis clients")

plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharex=True)
fig.suptitle('Description de la distance en kilomètres entre clients et vendeurs')

sns.histplot(ax=axes[0],x="distance_customer_seller", data=data, kde=True)
axes[0].set_title("Distribution de la distance en kilomètres entre clients et vendeurs")

sns.boxplot(ax=axes[1], x="distance_customer_seller", data=data)
axes[1].set_title("Boxplot de la distance en kilomètres entre clients et vendeurs")

plt.show()

In [ ]:
data.drop("nb_days_before_delivered_estimation", axis=1, inplace=True)

In [ ]:
data.to_csv("datasets/data_clustering_complete.csv")

**Ressources :**

- Intégration de la plateforme Olist sur le site de Carrefour Brésil: https://www.carrefour.com.br/parceiro/olist